Code for extracting super-voxel ROIS from the Keller/Zlatic data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import pyspark

from janelia_core.fileio.exp_reader import read_img_file
from keller_zlatic_vnc.raw_data_processing import video_to_supervoxel_baselines

## Parameters go here

In [3]:
# Location of excel file specifying where the data for each experiment is saved relative to the base folder
data_loc_file = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Parameters for where the brain mask is located
brain_mask_folder = r'A:\\projects\\keller_vnc\\data\\registered_brain_mask'
brain_mask_file = 'thresholdedVoxelMapSmooth.klb'

# Name of the subfolder we create under the [Main Folder]\[Subfolder]\ directory to save extracted results
extracted_folder = 'extracted\\rois'

# Additional parameters (constant for all datasets) specifying where the data is stored
image_base_folder =r'K:\\SV4'
image_processed_folder = 'Results\\WeightFused'
image_ext = r'weightFused.TimeRegistration.templateSpace.klb'

# Parameters for how we extract supervoxels
roi_extract_opts = {'voxel_size_per_dim':[1, 10, 10], 'brain_mask_perc': .75}

# Parameters for calculating baseline
baseline_calc_opts = {'window_length': 41, 'filter_start': -20, 'write_offset': 20, 'p': .1, 'n_processes': 80}

## Read in excel file specifying location of each dataset

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(data_loc_file, header=1, usecols=[1, 2], converters=converters)

## Load the brain mask

In [5]:
brain_mask_file = Path(brain_mask_folder) / brain_mask_file
brain_mask = read_img_file(brain_mask_file)

## Create a spark Context

In [6]:
conf = pyspark.SparkConf().setMaster('local[20]').setAll([
    ('spark.executor.memory', '10g'), ('spark.driver.memory','400g'), ('spark.driver.maxResultSize', '300g')])
sc = pyspark.SparkContext(conf=conf)

## Process each dataset

In [7]:
roi_extract_opts['brain_mask'] = brain_mask

In [10]:
for d_i in range(1):
    print('**********************************************************************************************')
    print('Processing subject')
    
    base_data_dir = Path(image_base_folder) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / image_processed_folder
    save_dir =  Path(image_base_folder) / data_locs['Main folder'][d_i] / data_locs['Subfolder'][d_i] / Path(extracted_folder)
    
    roi_info = video_to_supervoxel_baselines(base_data_dir=base_data_dir, save_dir=save_dir, roi_extract_opts=roi_extract_opts, 
                                       baseline_calc_opts=baseline_calc_opts, new_comp=False, sc=sc)

**********************************************************************************************
Processing subject
Save directory already exists: K:\SV4\CW_17-08-23\L1-561nm-ROIMonitoring_20170823_145226.corrected\extracted\rois
Beginning supervoxel extraction.
Searching for image files...
Found 5416 images.
Extracting: 94914 super voxels from 5416 images.
Extracted 94914 ROIS in 776.4003269672394 seconds.
Beginning baseline calculation.
Baselines calculated in 384.95537877082825 seconds.
